In [1]:
from pandas import DataFrame, read_csv
import random
import math

In [2]:
random.gauss(0, 0.5)

-0.4748250020686569

In [3]:
paramsToStore = {"Time": [0,1,2,3,0,1,2,3,4,5],
                "TTC": [random.uniform(0, 10) for _ in range(10)],
                "DTO": [random.uniform(0, 20) for _ in range(10)],
                "JERK": [abs(random.gauss(0, 20)) for _ in range(10)],
                "Speed": [random.uniform(0, 15) for _ in range(10)],
                "AngularSpeedX": [random.gauss(0, 0.15) for _ in range(10)],
                "AngularSpeedY": [random.gauss(0, 0.15) for _ in range(10)],
                "AngularSpeedZ": [random.gauss(0, 0.1) for _ in range(10)],
                "COL": [0,0,0,1,0,0,0,0,1,0]}
df = DataFrame(paramsToStore)
# df.set_index("Time", inplace=True)
# df.to_csv(PATH+"/data/testGenerateData.csv", index=False)
df.head(10)

,Time,TTC,DTO,JERK,Speed,AngularSpeedX,AngularSpeedY,AngularSpeedZ,COL
0,0,9.793003,7.761143,6.290391,7.652127,-0.208373,-0.000183,-0.139520,0
1,1,6.027473,12.133954,20.704108,10.805318,0.076245,-0.285912,0.176318,0
2,2,3.918056,4.172877,0.734836,9.625467,-0.171730,0.030075,0.039270,0
3,3,6.348410,9.777261,7.325581,11.395948,-0.002795,-0.364585,0.038641,1
4,0,2.108651,7.948898,20.936127,14.765548,-0.132504,-0.001296,0.036218,0
5,1,8.886117,16.886996,5.734412,2.641346,0.188798,-0.188439,0.040880,0
6,2,1.457497,3.889837,33.269987,0.036322,-0.112036,0.187535,0.035712,0
7,3,1.934603,1.564641,0.721339,7.460422,0.031294,0.167389,0.027868,0
8,4,2.996611,16.410155,3.204516,3.400765,0.273853,-0.355757,-0.221490,1
9,5,5.048916,2.245232,15.966939,1.884696,-0.346260,0.068548,0.001533,0


In [4]:
l = [0]
l[-3:] = [1]*3
print(l)

[1, 1, 1]


In [5]:
def makeDataUsable(df: DataFrame, pastImportance: int=6, secBeforeCol: int=3) -> DataFrame:
    """
    Make a usable Dataframe from a csv file from generated data.\\
    Creates pastImportance of columns for each prediction feature.\\
    Registers the "col" feature before an actual collision as 1, up to secBeforeCol back in time.
    
    ### Params:
        * df: Dataframe, need to have the columns named "TTC", "DTO", "JERK" and "Speed"ArithmeticError
        * pastImportance: int, needs to be > 0
        * secBeforeCol: int, needs to be > 0
    ### Returns:
        * Dataframe
    """
    colsToUse = ["TTC", "DTO", "JERK", "Speed", "asX", "asY", "asZ"]
    columns = ["Time"]
    columns += [f"{c}{i}" for c in colsToUse for i in range(1, pastImportance+1)]
    columns += ["COL"]
    
    # colsToUse = list(df.columns)
    # columns = colsToUse[0]
    # columns += [f"{c}{i}" for i in range(1, pastImportance+1) for c in colsToUse[1:-1]]
    # columns = colsToUse[-1]
    
    dataDict = {col: [] for col in columns}

    for i, row in df.iterrows():
        if row["Time"] < pastImportance-1:
        # if row[colsToUse[0]] < pastImportance-1:   
            continue

        dataDict["Time"].append(row["Time"])
        # dataDict[colsToUse[0]].append(row[colsToUse[0]])
        for j, k in enumerate(range(i-pastImportance+1, i+1), start=1):
            for c in colsToUse:
            # for c in colsToUse[1:-1]:
                dataDict[f"{c}{j}"].append(df.iloc[k][c])

        dataDict[f"COL"].append(row["COL"])
        if row["COL"] == 1:
            available = int(row["Time"]-pastImportance+2)
            amount = int(secBeforeCol) if available >= secBeforeCol else available
            dataDict["COL"][-amount:] = [1]*amount

    # for key, val in dataDict.items():
    #     print(key, val, len(val))
    return DataFrame(dataDict)

# makeDataUsable(df, 3, 3)
makeDataUsable(read_csv("../../data/testGenerateData.csv"),4, 3)

,Time,TTC1,TTC2,TTC3,TTC4,DTO1,DTO2,DTO3,DTO4,JERK1,...,asX4,asY1,asY2,asY3,asY4,asZ1,asZ2,asZ3,asZ4,COL
0,3.0,50.000,50.0,50.0,50.000,39.2822,100.0000,100.0000,100.0000,0.013,...,0.000,-0.000,-0.000,-0.001,-0.001,-0.004,0.006,0.001,0.000,0.0
1,4.0,50.000,50.0,50.0,50.000,100.0000,100.0000,100.0000,100.0000,0.661,...,-0.001,-0.000,-0.001,-0.001,0.000,0.006,0.001,0.000,-0.001,0.0
2,5.0,50.000,50.0,50.0,50.000,100.0000,100.0000,100.0000,100.0000,4.117,...,-0.000,-0.001,-0.001,0.000,0.000,0.001,0.000,-0.001,0.000,0.0
3,6.0,50.000,50.0,50.0,50.000,100.0000,100.0000,100.0000,100.0000,0.865,...,0.000,-0.001,0.000,0.000,0.000,0.000,-0.001,0.000,0.000,0.0
4,7.0,50.000,50.0,50.0,5.322,100.0000,100.0000,100.0000,83.3296,3.704,...,0.004,0.000,0.000,0.000,-0.001,-0.001,0.000,0.000,0.005,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,297.0,0.024,50.0,50.0,50.000,0.3569,2.4363,4.5318,5.3807,10.762,...,0.004,-0.001,-0.000,-0.000,0.302,-0.117,-0.001,-0.000,0.004,0.0
295,298.0,50.000,50.0,50.0,50.000,2.4363,4.5318,5.3807,17.0737,11.018,...,-0.006,-0.000,-0.000,0.302,0.005,-0.001,-0.000,0.004,-0.004,0.0
296,299.0,50.000,50.0,50.0,50.000,4.5318,5.3807,17.0737,100.0000,0.000,...,0.001,-0.000,0.302,0.005,-0.504,-0.000,0.004,-0.004,-0.013,0.0
297,300.0,50.000,50.0,50.0,50.000,5.3807,17.0737,100.0000,34.4654,0.818,...,0.016,0.302,0.005,-0.504,-0.033,0.004,-0.004,-0.013,0.007,0.0


In [13]:
data = makeDataUsable(read_csv("../../data/testGenerateData.csv"), 6, 4)
data.head(10)


,Time,TTC1,TTC2,TTC3,TTC4,TTC5,TTC6,DTO1,DTO2,DTO3,...,asY4,asY5,asY6,asZ1,asZ2,asZ3,asZ4,asZ5,asZ6,COL
0,5.0,50.000,50.000,50.000,50.000,50.000,50.000,39.2822,100.0000,100.0000,...,-0.001,0.000,0.000,-0.004,0.006,0.001,0.000,-0.001,0.000,0.0
1,6.0,50.000,50.000,50.000,50.000,50.000,50.000,100.0000,100.0000,100.0000,...,0.000,0.000,0.000,0.006,0.001,0.000,-0.001,0.000,0.000,0.0
2,7.0,50.000,50.000,50.000,50.000,50.000,5.322,100.0000,100.0000,100.0000,...,0.000,0.000,-0.001,0.001,0.000,-0.001,0.000,0.000,0.005,0.0
3,8.0,50.000,50.000,50.000,50.000,5.322,4.585,100.0000,100.0000,100.0000,...,0.000,-0.001,0.000,0.000,-0.001,0.000,0.000,0.005,-0.000,0.0
4,9.0,50.000,50.000,50.000,5.322,4.585,0.469,100.0000,100.0000,100.0000,...,-0.001,0.000,0.655,-0.001,0.000,0.000,0.005,-0.000,0.041,0.0
5,10.0,50.000,50.000,5.322,4.585,0.469,2.421,100.0000,100.0000,83.3296,...,0.000,0.655,-0.974,0.000,0.000,0.005,-0.000,0.041,0.218,0.0
6,11.0,50.000,5.322,4.585,0.469,2.421,50.000,100.0000,83.3296,67.9841,...,0.655,-0.974,0.994,0.000,0.005,-0.000,0.041,0.218,-0.094,0.0
7,12.0,5.322,4.585,0.469,2.421,50.000,50.000,83.3296,67.9841,22.0613,...,-0.974,0.994,0.135,0.005,-0.000,0.041,0.218,-0.094,0.023,0.0
8,13.0,4.585,0.469,2.421,50.000,50.000,2.238,67.9841,22.0613,15.8667,...,0.994,0.135,-0.512,-0.000,0.041,0.218,-0.094,0.023,0.020,0.0
9,14.0,0.469,2.421,50.000,50.000,2.238,50.000,22.0613,15.8667,100.0000,...,0.135,-0.512,-0.001,0.041,0.218,-0.094,0.023,0.020,0.001,0.0


In [14]:
data.corr()["COL"]

Time     -0.100039
TTC1      0.110867
TTC2      0.008383
TTC3     -0.156876
TTC4     -0.293138
TTC5     -0.395535
TTC6     -0.473721
DTO1      0.033025
DTO2      0.016898
DTO3     -0.090762
DTO4     -0.179973
DTO5     -0.297591
DTO6     -0.384443
JERK1    -0.027663
JERK2    -0.086778
JERK3    -0.108736
JERK4    -0.125656
JERK5    -0.168536
JERK6     0.112899
Speed1    0.039142
Speed2    0.105527
Speed3    0.169866
Speed4    0.211131
Speed5    0.235813
Speed6    0.034796
asX1     -0.043545
asX2      0.071011
asX3      0.066579
asX4      0.046845
asX5      0.040693
asX6      0.062918
asY1      0.004007
asY2     -0.020697
asY3     -0.010802
asY4     -0.024909
asY5     -0.004092
asY6      0.014476
asZ1     -0.076694
asZ2      0.044286
asZ3      0.020662
asZ4      0.016362
asZ5      0.034749
asZ6      0.018157
COL       1.000000
Name: COL, dtype: float64

In [15]:
from useGeneratedData import makeDataUsable, splitTrainTest, NewPredicter
from pandas import DataFrame, read_csv

In [16]:
# data = makeDataUsable(read_csv("../../data/testGenerateData.csv"), 6, 4)
dataToUse = data.copy()
dataToUse.drop(columns=data.columns[0], axis=1, inplace=True)
# print(data.head(10))
trainX, trainY, testX, testY = splitTrainTest(dataToUse)
print(f"trainX:{trainX.shape}, trainY:{trainY.shape}, testX:{testX.shape}, testY:{testY.shape}")
# print(trainX.head())
print(f"trainX: {type(trainX)}, trainY: {type(trainY)}, testX: {type(testX)}, testY: {type(testY)}")


splitting at 215.
trainX:(215, 42), trainY:(215,), testX:(54, 42), testY:(54,)
trainX: <class 'pandas.core.frame.DataFrame'>, trainY: <class 'pandas.core.series.Series'>, testX: <class 'pandas.core.frame.DataFrame'>, testY: <class 'pandas.core.series.Series'>


In [17]:

# p = NewPredicter.loadModel("xgb_2_582-11-16-201")
# print(p.__dict__)

p = NewPredicter()
trainXpp = p.preProcess(trainX)
trainYpp = trainY.to_numpy()
testXpp = p.preProcess(testX)
testYpp = testY.to_numpy()
print(f"trainX: {type(trainXpp)}, trainY: {type(trainYpp)}, testX: {type(testXpp)}, testY: {type(testYpp)}")
# print(trainXpp)
p.fit(trainXpp, trainYpp)

# p.predict([])

Scaler is fitted
trainX: <class 'numpy.ndarray'>, trainY: <class 'numpy.ndarray'>, testX: <class 'numpy.ndarray'>, testY: <class 'numpy.ndarray'>


In [18]:
pred = []
for i in range(len(testX)):
    pred.append(p.predict(testX.iloc[i]))
print(pred)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]


In [19]:
p.getScore(pred, testY)

Total: 54, number of collisions: 14
	TN: 36 	| FP: 4 
	FN: 4 	| TP: 10
Accuracy: 0.85
Precision: 0.71
Recall: 0.71
F1: 0.71


[[36, 4], [4, 10]]